# Import Package

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import sklearn

# Preprocessing

In [2]:
def outlier_preprocess(df, price, floor, area):
    df = df[df['單價'] != price]
    df = df[df['總樓層數'] < floor]
    df = df[df['建物面積'] < area].reset_index(drop=True)
    
    return df

In [3]:
def missing_value(df,*values):
    
    for i in range(len(est_list)):
        df.loc[df['鄉鎮市區'] == est_list[i], '單價'] = values[i]
    return df

In [4]:
def parking_space(df,*values):
    for i in range(int(df['車位個數'].max())+1):
        if i == 0:
            df.loc[df['車位個數'] == i, '車位面積'] = values[i]
        elif i == 2:
            df.loc[(df['車位個數'] == i) & (df['車位面積'] == df['車位面積'].min()), '車位面積'] = values[i]
        else:
            pass
    return df

In [5]:
def same_obj(df):
        
    for dist, road, floor, btype, use in zip(df['鄉鎮市區'], df['路名'], df['總樓層數'],
                                         df['建物型態'], df['主要用途']):
        loc_condition = (df['鄉鎮市區'] == dist) & \
                        (df['路名'] == road) & \
                        (df['總樓層數'] == floor) & \
                        (df['建物型態'] == btype) & \
                        (df['主要用途'] == use)
    
        if (len(df[loc_condition]) > 1):
            if (df[loc_condition]['屋齡'].max() - df[loc_condition]['屋齡'].min()) < 1:
            
                df.loc[loc_condition, '橫坐標'] = df.loc[loc_condition, '橫坐標'].mean()
                df.loc[loc_condition, '縱坐標'] = df.loc[loc_condition, '縱坐標'].mean()
            else:
                pass
        else:
            pass
                       
    return df

In [6]:
# Load Data
Train = pd.read_csv('C:\\Users\\USER\\house_price\\30_Training Dataset_V2\\training_data.csv')
Public = pd.read_csv('C:\\Users\\USER\\house_price\\30_Public Dataset_Public Sumission Template_v2\\public_dataset.csv')
Private = pd.read_csv('C:\\Users\\USER\\house_price\\30_Private Dataset _Private and Publict Submission Template_v2\\private_dataset.csv')
housebroker = pd.read_excel(r'C:\Users\USER\house_price\30_Training Dataset_V2\external_data\broker.xlsx')

In [7]:
# 確認訓練資料及缺失值(鄉鎮市區出現於測試資料及但未存在於訓練資料集)
est_list = \
list(Public[~(Public['鄉鎮市區'].isin(Train['鄉鎮市區']))]['鄉鎮市區'].drop_duplicates(keep='first')) + \
list(Private[(~(Private['鄉鎮市區'].isin(Train['鄉鎮市區']))) & 
             (~(Private['鄉鎮市區'].isin(Public['鄉鎮市區'])))]
     ['鄉鎮市區'].drop_duplicates(keep='first')
    )

In [8]:
df = pd.concat([Train, Public[Public['鄉鎮市區'].isin(est_list)], Private[Private['鄉鎮市區'].isin(est_list)]]).reset_index(drop=True)

In [9]:
pivot_housebroker = pd.DataFrame(housebroker.pivot_table(index = '鄉鎮市區', aggfunc='size')).dropna().reset_index()
#pivot_housebroker.rename(columns ={ 0:'count'})
df = df.merge(pivot_housebroker, on = '鄉鎮市區', how = 'left').rename(columns={0:'count'})

In [10]:
# Preprocessing
df = outlier_preprocess(df,df['單價'].max(),50,6)

df = missing_value(df,
                   df[df['縣市'] == '高雄市']['單價'].mean(),
                   df[df['縣市'] == '苗栗縣']['單價'].mean(),
                   df[df['縣市'] == '桃園市']['單價'].quantile(0.3),
                   df[df['縣市'] == '新北市']['單價'].quantile(0.2),
                   df[df['縣市'] == '台中市']['單價'].quantile(0.2),
                   df[df['縣市'] == '台南市']['單價'].quantile(0.2),
                   df[df['縣市'] == '苗栗縣']['單價'].mean(),
                   df[df['縣市'] == '高雄市']['單價'].quantile(0.25)
                  )

df = parking_space(df,
                   df['車位面積'].min(),
                   None,
                   df[df['車位個數'] == 2]['車位面積'].quantile(0.05),
                   None
                  )
                
df = same_obj(df)



C:\Users\USER\AppData\Local\Temp\ipykernel_16336\1995322956.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '296638.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[loc_condition, '橫坐標'] = df.loc[loc_condition, '橫坐標'].mean()
C:\Users\USER\AppData\Local\Temp\ipykernel_16336\1995322956.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2772503.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[loc_condition, '縱坐標'] = df.loc[loc_condition, '縱坐標'].mean()


# loc_df

In [11]:
# 參考開源資源經緯度轉換公式
def twd97_to_lonlat(x=174458.0,y=2525824.0):
    """
    Parameters
    ----------
    x : float
        TWD97 coord system. The default is 174458.0.
    y : float
        TWD97 coord system. The default is 2525824.0.
    Returns
    -------
    list
        [longitude, latitude]
    """
    
    a = 6378137
    b = 6356752.314245
    long_0 = 121 * math.pi / 180.0
    k0 = 0.9999
    dx = 250000
    dy = 0
    
    e = math.pow((1-math.pow(b, 2)/math.pow(a,2)), 0.5)
    
    x -= dx
    y -= dy
    
    M = y / k0
    
    mu = M / ( a*(1-math.pow(e, 2)/4 - 3*math.pow(e,4)/64 - 5 * math.pow(e, 6)/256))
    e1 = (1.0 - pow((1   - pow(e, 2)), 0.5)) / (1.0 +math.pow((1.0 -math.pow(e,2)), 0.5))
    
    j1 = 3*e1/2-27*math.pow(e1,3)/32
    j2 = 21 * math.pow(e1,2)/16 - 55 * math.pow(e1, 4)/32
    j3 = 151 * math.pow(e1, 3)/96
    j4 = 1097 * math.pow(e1, 4)/512
    
    fp = mu + j1 * math.sin(2*mu) + j2 * math.sin(4* mu) + j3 * math.sin(6*mu) + j4 * math.sin(8* mu)
    
    e2 = math.pow((e*a/b),2)
    c1 = math.pow(e2*math.cos(fp),2)
    t1 = math.pow(math.tan(fp),2)
    r1 = a * (1-math.pow(e,2)) / math.pow( (1-math.pow(e,2)* math.pow(math.sin(fp),2)), (3/2))
    n1 = a / math.pow((1-math.pow(e,2)*math.pow(math.sin(fp),2)),0.5)
    d = x / (n1*k0)
    
    q1 = n1* math.tan(fp) / r1
    q2 = math.pow(d,2)/2
    q3 = ( 5 + 3 * t1 + 10 * c1 - 4 * math.pow(c1,2) - 9 * e2 ) * math.pow(d,4)/24
    q4 = (61 + 90 * t1 + 298 * c1 + 45 * math.pow(t1,2) - 3 * math.pow(c1,2) - 252 * e2) * math.pow(d,6)/720
    lat = fp - q1 * (q2 - q3 + q4)
    
    
    q5 = d
    q6 = (1+2*t1+c1) * math.pow(d,3) / 6
    q7 = (5 - 2 * c1 + 28 * t1 - 3 * math.pow(c1,2) + 8 * e2 + 24 * math.pow(t1,2)) * math.pow(d,5) / 120
    lon = long_0 + (q5 - q6 + q7) / math.cos(fp)
    
    lat = (lat*180) / math.pi
    lon = (lon*180) / math.pi
    return [lon, lat]

In [12]:
def loc_preprocess(df,lng_col_name,lat_col_name):
    x_loc = []
    for lon_value, lat_value in zip(df[str(lng_col_name)], df[str(lat_col_name)]):
        values = twd97_to_lonlat(lon_value, lat_value)
        x_loc.append(values)
    loc_df = pd.DataFrame(x_loc, columns = ['lng', 'lat'])
    return loc_df

In [13]:
def distance(loc_df,external_df,col_name):
    distance_min = []
    distance_density = []
    criteria = 0.55
    for lng, lat in zip(loc_df['lng'], loc_df['lat']):
        distance = np.min(np.abs(lng - external_df['lng']) + np.abs(lat - external_df['lat']))
        distance_min.append(distance)
        
        distance_d = np.abs(lng - external_df['lng']) + np.abs(lat - external_df['lat'])
        sum_ = sum(distance_d[(distance_d < criteria)])
        distance_density.append(sum_)

    distance_min = pd.DataFrame(distance_min, columns = [str(col_name)+'_min'])
    distance_density = pd.DataFrame(distance_density, columns = [str(col_name)+'_density'])
    return pd.concat([distance_min, distance_density],axis=1)

In [14]:
#Load External Data
MRT = pd.read_csv(r'C:\Users\USER\house_price\30_Training Dataset_V2\external_data\捷運站點資料.csv')
FIN = pd.read_csv(r'C:\Users\USER\house_price\30_Training Dataset_V2\external_data\金融機構基本資料.csv')
Dep = pd.read_csv(r'C:\Users\USER\house_price\30_Training Dataset_V2\external_data\dep_loc_df.csv')
University = pd.read_csv(r'C:\Users\USER\house_price\30_Training Dataset_V2\external_data\大學基本資料.csv')
broker = pd.read_excel(r'C:\Users\USER\house_price\30_Training Dataset_V2\external_data\broker_loc_df.xlsx')
park = pd.read_csv(r'C:\Users\USER\house_price\30_Training Dataset_V2\external_data\IPD25598.csv')

In [15]:
loc_df = loc_preprocess(df,'橫坐標','縱坐標')
park = park[(park['類別'] == '科學園區') | (park['類別'] == '科技產業園區')].reset_index(drop=True)
park = loc_preprocess(park,'X','Y')

In [16]:
MRT_distance = distance(loc_df,MRT,'MRT')
FIN_distance = distance(loc_df,FIN,'FIN')
Dep_distance = distance(loc_df,Dep,'Dep')
University_distance = distance(loc_df,University,'University')
broker_distance = distance(loc_df,broker,'broker')
park_distance = distance(loc_df,park,'park')
# location train data
x_loc_df = pd.concat([MRT_distance,
                      FIN_distance,
                      Dep_distance,
                      University_distance,
                      broker_distance,
                      park_distance], axis=1)

# Categorical

In [17]:
def encoder(df_fit,df,fit=True,*col_names):
    ohe = OneHotEncoder(sparse_output = False)
    
    if fit == True:
        X_data = pd.concat([df_fit[col] for col in col_names], axis=1)
        X = ohe.fit_transform(X_data)
        encoded_column_names = ohe.get_feature_names_out()
        encoded_column_names = pd.DataFrame(encoded_column_names)
        x_cat_set = pd.DataFrame(X, columns = encoded_column_names.iloc[:, 0])
    else:
        X_data = pd.concat([df_fit[col] for col in col_names], axis=1)
        X = ohe.fit_transform(X_data)
        encoded_column_names = ohe.get_feature_names_out()
        encoded_column_names = pd.DataFrame(encoded_column_names)
        
        X_data_test = pd.concat([df[col] for col in col_names], axis=1)
        X_test = ohe.transform(X_data_test)
        encoded_column_names_test = ohe.get_feature_names_out()
        encoded_column_names_test = pd.DataFrame(encoded_column_names_test)
        x_cat_set = pd.DataFrame(X_test, columns = encoded_column_names.iloc[:, 0])
        
    
    return x_cat_set

In [18]:
x_cat_df = encoder(df,df,True,'縣市','鄉鎮市區','建物型態')

# Numerical

In [19]:
def numerical(df,*col_names):
    x_num_set = pd.concat([df[col] for col in col_names], axis=1)
    return x_num_set

In [20]:
x_num_df = numerical(df,'屋齡','車位面積','建物面積','移轉層次','總樓層數','橫坐標','縱坐標','count','陽台面積','附屬建物面積')

# Training

In [21]:
X_train_df = pd.concat([x_loc_df,x_cat_df,x_num_df],axis=1)
y_train_df = df[['單價']]

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_df, y_train_df, random_state = 1)

In [23]:
# Fine Tuning
'''
param_space = {
    'colsample_bytree': Real(0.5, 1, prior='uniform'),
    'gamma': Real(0.000000001, 0.01, prior='uniform'),
    'learning_rate': Real(0.000001, 0.01, prior='log-uniform'),
    'max_depth': Integer(20, 100),
    'min_child_weight': Integer(1, 15, prior = 'uniform'),
    'n_estimators': Integer(4000, 8500),
    'subsample': Real(0.5, 1.0, prior='uniform'),
}
opt = BayesSearchCV(
    estimator=xgb.XGBRegressor(),
    search_spaces=param_space,
    n_iter=50,  
    scoring='neg_mean_absolute_error', 
    n_jobs=-1, 
    cv=5,  
)
opt.fit(X_train, y_train)

best_params = opt.best_params_
'''

"\nparam_space = {\n    'colsample_bytree': Real(0.5, 1, prior='uniform'),\n    'gamma': Real(0.000000001, 0.01, prior='uniform'),\n    'learning_rate': Real(0.000001, 0.01, prior='log-uniform'),\n    'max_depth': Integer(20, 100),\n    'min_child_weight': Integer(1, 15, prior = 'uniform'),\n    'n_estimators': Integer(4000, 8500),\n    'subsample': Real(0.5, 1.0, prior='uniform'),\n}\nopt = BayesSearchCV(\n    estimator=xgb.XGBRegressor(),\n    search_spaces=param_space,\n    n_iter=50,  \n    scoring='neg_mean_absolute_error', \n    n_jobs=-1, \n    cv=5,  \n)\nopt.fit(X_train, y_train)\n\nbest_params = opt.best_params_\n"

In [24]:
# 模型訓練
xgb_model = XGBRegressor(colsample_bytree = 0.7047123824101982,
                         gamma = 1e-07,
                         learning_rate = 0.001403915766812771,
                         max_depth = 93,
                         min_child_weight = 8,
                         n_estimators = 6500,
                         subsample = 0.5103054506923898,
                         eval_metric = 'mape',
                         device='cuda:0',
                         random_state = 5)

xgb_model.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_valid,y_valid)])

[0]	validation_0-mape:0.42820	validation_1-mape:0.43080
[1]	validation_0-mape:0.42765	validation_1-mape:0.43026
[2]	validation_0-mape:0.42712	validation_1-mape:0.42973
[3]	validation_0-mape:0.42659	validation_1-mape:0.42922
[4]	validation_0-mape:0.42605	validation_1-mape:0.42869
[5]	validation_0-mape:0.42551	validation_1-mape:0.42816
[6]	validation_0-mape:0.42498	validation_1-mape:0.42764
[7]	validation_0-mape:0.42446	validation_1-mape:0.42713
[8]	validation_0-mape:0.42393	validation_1-mape:0.42660
[9]	validation_0-mape:0.42339	validation_1-mape:0.42608
[10]	validation_0-mape:0.42287	validation_1-mape:0.42556
[11]	validation_0-mape:0.42235	validation_1-mape:0.42505
[12]	validation_0-mape:0.42181	validation_1-mape:0.42452
[13]	validation_0-mape:0.42128	validation_1-mape:0.42400
[14]	validation_0-mape:0.42076	validation_1-mape:0.42349
[15]	validation_0-mape:0.42024	validation_1-mape:0.42298
[16]	validation_0-mape:0.41973	validation_1-mape:0.42247
[17]	validation_0-mape:0.41919	validation

[144]	validation_0-mape:0.35780	validation_1-mape:0.36178
[145]	validation_0-mape:0.35735	validation_1-mape:0.36134
[146]	validation_0-mape:0.35691	validation_1-mape:0.36091
[147]	validation_0-mape:0.35647	validation_1-mape:0.36048
[148]	validation_0-mape:0.35604	validation_1-mape:0.36006
[149]	validation_0-mape:0.35560	validation_1-mape:0.35964
[150]	validation_0-mape:0.35516	validation_1-mape:0.35921
[151]	validation_0-mape:0.35473	validation_1-mape:0.35878
[152]	validation_0-mape:0.35429	validation_1-mape:0.35835
[153]	validation_0-mape:0.35386	validation_1-mape:0.35793
[154]	validation_0-mape:0.35342	validation_1-mape:0.35751
[155]	validation_0-mape:0.35298	validation_1-mape:0.35708
[156]	validation_0-mape:0.35254	validation_1-mape:0.35665
[157]	validation_0-mape:0.35210	validation_1-mape:0.35622
[158]	validation_0-mape:0.35166	validation_1-mape:0.35579
[159]	validation_0-mape:0.35123	validation_1-mape:0.35536
[160]	validation_0-mape:0.35079	validation_1-mape:0.35493
[161]	validati

[286]	validation_0-mape:0.30090	validation_1-mape:0.30634
[287]	validation_0-mape:0.30053	validation_1-mape:0.30599
[288]	validation_0-mape:0.30017	validation_1-mape:0.30564
[289]	validation_0-mape:0.29981	validation_1-mape:0.30529
[290]	validation_0-mape:0.29944	validation_1-mape:0.30493
[291]	validation_0-mape:0.29907	validation_1-mape:0.30458
[292]	validation_0-mape:0.29871	validation_1-mape:0.30422
[293]	validation_0-mape:0.29835	validation_1-mape:0.30387
[294]	validation_0-mape:0.29799	validation_1-mape:0.30352
[295]	validation_0-mape:0.29763	validation_1-mape:0.30317
[296]	validation_0-mape:0.29727	validation_1-mape:0.30283
[297]	validation_0-mape:0.29692	validation_1-mape:0.30249
[298]	validation_0-mape:0.29656	validation_1-mape:0.30214
[299]	validation_0-mape:0.29620	validation_1-mape:0.30180
[300]	validation_0-mape:0.29584	validation_1-mape:0.30145
[301]	validation_0-mape:0.29548	validation_1-mape:0.30110
[302]	validation_0-mape:0.29512	validation_1-mape:0.30075
[303]	validati

[428]	validation_0-mape:0.25416	validation_1-mape:0.26118
[429]	validation_0-mape:0.25386	validation_1-mape:0.26089
[430]	validation_0-mape:0.25356	validation_1-mape:0.26061
[431]	validation_0-mape:0.25326	validation_1-mape:0.26033
[432]	validation_0-mape:0.25298	validation_1-mape:0.26005
[433]	validation_0-mape:0.25269	validation_1-mape:0.25978
[434]	validation_0-mape:0.25240	validation_1-mape:0.25951
[435]	validation_0-mape:0.25211	validation_1-mape:0.25922
[436]	validation_0-mape:0.25181	validation_1-mape:0.25894
[437]	validation_0-mape:0.25151	validation_1-mape:0.25865
[438]	validation_0-mape:0.25121	validation_1-mape:0.25837
[439]	validation_0-mape:0.25093	validation_1-mape:0.25810
[440]	validation_0-mape:0.25063	validation_1-mape:0.25782
[441]	validation_0-mape:0.25034	validation_1-mape:0.25754
[442]	validation_0-mape:0.25005	validation_1-mape:0.25725
[443]	validation_0-mape:0.24976	validation_1-mape:0.25697
[444]	validation_0-mape:0.24948	validation_1-mape:0.25670
[445]	validati

[570]	validation_0-mape:0.21599	validation_1-mape:0.22494
[571]	validation_0-mape:0.21575	validation_1-mape:0.22471
[572]	validation_0-mape:0.21550	validation_1-mape:0.22448
[573]	validation_0-mape:0.21526	validation_1-mape:0.22425
[574]	validation_0-mape:0.21502	validation_1-mape:0.22402
[575]	validation_0-mape:0.21477	validation_1-mape:0.22380
[576]	validation_0-mape:0.21454	validation_1-mape:0.22359
[577]	validation_0-mape:0.21431	validation_1-mape:0.22336
[578]	validation_0-mape:0.21407	validation_1-mape:0.22314
[579]	validation_0-mape:0.21383	validation_1-mape:0.22292
[580]	validation_0-mape:0.21359	validation_1-mape:0.22270
[581]	validation_0-mape:0.21335	validation_1-mape:0.22247
[582]	validation_0-mape:0.21311	validation_1-mape:0.22224
[583]	validation_0-mape:0.21287	validation_1-mape:0.22202
[584]	validation_0-mape:0.21263	validation_1-mape:0.22179
[585]	validation_0-mape:0.21239	validation_1-mape:0.22156
[586]	validation_0-mape:0.21216	validation_1-mape:0.22134
[587]	validati

...

[5999]	validation_0-mape:0.02435	validation_1-mape:0.08578
[6000]	validation_0-mape:0.02434	validation_1-mape:0.08578
[6001]	validation_0-mape:0.02434	validation_1-mape:0.08578
[6002]	validation_0-mape:0.02433	validation_1-mape:0.08578
[6003]	validation_0-mape:0.02433	validation_1-mape:0.08578
[6004]	validation_0-mape:0.02432	validation_1-mape:0.08578
[6005]	validation_0-mape:0.02432	validation_1-mape:0.08578
[6006]	validation_0-mape:0.02431	validation_1-mape:0.08578
[6007]	validation_0-mape:0.02431	validation_1-mape:0.08578
[6008]	validation_0-mape:0.02431	validation_1-mape:0.08578
[6009]	validation_0-mape:0.02430	validation_1-mape:0.08578
[6010]	validation_0-mape:0.02430	validation_1-mape:0.08578
[6011]	validation_0-mape:0.02429	validation_1-mape:0.08578
[6012]	validation_0-mape:0.02429	validation_1-mape:0.08578
[6013]	validation_0-mape:0.02428	validation_1-mape:0.08577
[6014]	validation_0-mape:0.02428	validation_1-mape:0.08577
[6015]	validation_0-mape:0.02427	validation_1-mape:0.085

[6138]	validation_0-mape:0.02371	validation_1-mape:0.08574
[6139]	validation_0-mape:0.02370	validation_1-mape:0.08574
[6140]	validation_0-mape:0.02370	validation_1-mape:0.08574
[6141]	validation_0-mape:0.02369	validation_1-mape:0.08574
[6142]	validation_0-mape:0.02369	validation_1-mape:0.08574
[6143]	validation_0-mape:0.02368	validation_1-mape:0.08574
[6144]	validation_0-mape:0.02368	validation_1-mape:0.08574
[6145]	validation_0-mape:0.02368	validation_1-mape:0.08574
[6146]	validation_0-mape:0.02367	validation_1-mape:0.08574
[6147]	validation_0-mape:0.02367	validation_1-mape:0.08574
[6148]	validation_0-mape:0.02366	validation_1-mape:0.08574
[6149]	validation_0-mape:0.02366	validation_1-mape:0.08574
[6150]	validation_0-mape:0.02365	validation_1-mape:0.08574
[6151]	validation_0-mape:0.02365	validation_1-mape:0.08574
[6152]	validation_0-mape:0.02364	validation_1-mape:0.08574
[6153]	validation_0-mape:0.02364	validation_1-mape:0.08574
[6154]	validation_0-mape:0.02364	validation_1-mape:0.085

[6277]	validation_0-mape:0.02308	validation_1-mape:0.08572
[6278]	validation_0-mape:0.02308	validation_1-mape:0.08572
[6279]	validation_0-mape:0.02307	validation_1-mape:0.08572
[6280]	validation_0-mape:0.02307	validation_1-mape:0.08572
[6281]	validation_0-mape:0.02306	validation_1-mape:0.08572
[6282]	validation_0-mape:0.02306	validation_1-mape:0.08572
[6283]	validation_0-mape:0.02306	validation_1-mape:0.08572
[6284]	validation_0-mape:0.02305	validation_1-mape:0.08572
[6285]	validation_0-mape:0.02305	validation_1-mape:0.08572
[6286]	validation_0-mape:0.02304	validation_1-mape:0.08572
[6287]	validation_0-mape:0.02304	validation_1-mape:0.08572
[6288]	validation_0-mape:0.02303	validation_1-mape:0.08572
[6289]	validation_0-mape:0.02303	validation_1-mape:0.08572
[6290]	validation_0-mape:0.02302	validation_1-mape:0.08572
[6291]	validation_0-mape:0.02302	validation_1-mape:0.08572
[6292]	validation_0-mape:0.02302	validation_1-mape:0.08572
[6293]	validation_0-mape:0.02301	validation_1-mape:0.085

[6416]	validation_0-mape:0.02248	validation_1-mape:0.08570
[6417]	validation_0-mape:0.02248	validation_1-mape:0.08570
[6418]	validation_0-mape:0.02247	validation_1-mape:0.08570
[6419]	validation_0-mape:0.02247	validation_1-mape:0.08570
[6420]	validation_0-mape:0.02247	validation_1-mape:0.08570
[6421]	validation_0-mape:0.02246	validation_1-mape:0.08570
[6422]	validation_0-mape:0.02246	validation_1-mape:0.08570
[6423]	validation_0-mape:0.02245	validation_1-mape:0.08570
[6424]	validation_0-mape:0.02245	validation_1-mape:0.08570
[6425]	validation_0-mape:0.02244	validation_1-mape:0.08570
[6426]	validation_0-mape:0.02244	validation_1-mape:0.08570
[6427]	validation_0-mape:0.02244	validation_1-mape:0.08570
[6428]	validation_0-mape:0.02243	validation_1-mape:0.08570
[6429]	validation_0-mape:0.02243	validation_1-mape:0.08570
[6430]	validation_0-mape:0.02242	validation_1-mape:0.08570
[6431]	validation_0-mape:0.02242	validation_1-mape:0.08570
[6432]	validation_0-mape:0.02241	validation_1-mape:0.085

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7047123824101982, device='cuda:0',
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='mape', feature_types=None, gamma=1e-07,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.001403915766812771,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=93, max_leaves=None,
             min_child_weight=8, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=6500, n_jobs=None,
             num_parallel_tree=None, random_state=5, ...)

# Evaluate

In [25]:
xgb_ev = xgb_model.predict(X_valid)
sklearn.metrics.mean_absolute_percentage_error(y_valid, xgb_ev)

C:\Users\USER\anaconda3\envs\DL\lib\site-packages\xgboost\core.py:160: UserWarning: [17:41:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


0.08569042459371594

# Submit

In [26]:
# Submit Data
df_test = pd.concat([Public,Private],axis=0).reset_index(drop=True)
df_test = df_test.merge(pivot_housebroker, on = '鄉鎮市區', how = 'left').rename(columns={0:'count'})

In [27]:
# Preprocessing

df_test = parking_space(df_test,
                   df_test['車位面積'].min(),
                   None,
                   df_test[df_test['車位個數'] == 2]['車位面積'].quantile(0.05),
                   None
                  )
                
df_test = same_obj(df_test)



C:\Users\USER\AppData\Local\Temp\ipykernel_16336\1995322956.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '288758.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[loc_condition, '橫坐標'] = df.loc[loc_condition, '橫坐標'].mean()
C:\Users\USER\AppData\Local\Temp\ipykernel_16336\1995322956.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2760282.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[loc_condition, '縱坐標'] = df.loc[loc_condition, '縱坐標'].mean()


In [28]:
loc_df = loc_preprocess(df_test,'橫坐標','縱坐標')
MRT_distance = distance(loc_df,MRT,'MRT')
FIN_distance = distance(loc_df,FIN,'FIN')
Dep_distance = distance(loc_df,Dep,'Dep')
University_distance = distance(loc_df,University,'University')
broker_distance = distance(loc_df,broker,'broker')
park_distance = distance(loc_df,park,'park')
# location train data
x_loc_df = pd.concat([MRT_distance,
                      FIN_distance,
                      Dep_distance,
                      University_distance,
                      broker_distance,
                      park_distance], axis=1)

In [29]:
x_cat_df = encoder(df,df_test,False,'縣市','鄉鎮市區','建物型態')

In [30]:
x_num_df = numerical(df_test,'屋齡','車位面積','建物面積','移轉層次','總樓層數','橫坐標','縱坐標','count','陽台面積','附屬建物面積')

In [31]:
X_test_df = pd.concat([x_loc_df, x_cat_df, x_num_df], axis = 1)

In [32]:
result_df = xgb_model.predict(X_test_df)
result_df = pd.DataFrame(result_df, columns = ['price'])
result_df.to_csv('result_df.csv', index=False, encoding='utf-8')